## Train model identify if two images are the same turtle

### Import libraries

In [ ]:
import os
import glob
import json
from os.path import join

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import random

from ShellRec.data_utils.prepare_photos import get_img_graph, split_graph
from ShellRec.training_utils.shell_rec import TurtleDiff, TurtleDataset, train

In [ ]:
random.seed(42)

### Preprocessing photo files

In [ ]:
all_train = get_img_graph(path = "../dataset/BoxTurtles")
split_graph(all_train, save_path = "../dataset")
_ = get_img_graph(path = "../dataset/BoxTurtlesHoldout", 
                  file_to_save = "../dataset/shell_rec_holdout.json")

### Prepare torch dataset

In [ ]:
## image transformations
transform_train = transforms.create_transform(384, is_training = True, 
                                   auto_augment = "rand-m9-mstd0.5")


transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4850, 0.4560, 0.4060], 
                             std=[0.2290, 0.2240, 0.2250])
])
    
# Set up datasets and dataloaders
train_loader = DataLoader( TurtleDataset(data_file='../dataset/train.json', 
                                             transform=transform_train), 
                                             batch_size = 20)
val_loader = DataLoader( TurtleDataset(data_file='../dataset/val.json', 
                                           transform=transform_test), 
                                           batch_size = 20)

### training

Get the model set up, use a pretrained model as backbone, and add a new head to it.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   

model = TurtleDiff('vit_base_patch16_384') # use a vit backbone
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
num_epochs = 10

Training loop

In [ ]:
train(model, optimizer, criterion, train_loader, val_loader, device, num_epochs,
      save_path = "../pretrained")